# Classifier Tests with extracted data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import extractor as ext

In [2]:
fbase = pd.read_csv('./1_filtered.csv', header=None)
fbase.head()

,0,1,2,3
0,8.456798,-4.466378,7.215951,0.019221
1,8.454815,-4.464132,7.212556,0.019295
2,8.453542,-4.461002,7.207107,0.019680
3,8.449433,-4.457170,7.201194,0.020556
4,8.440568,-4.453560,7.195996,0.020409


In [3]:
extractor = ext.RecordExtractor()
cycles = extractor.read_sample(fbase)
len(cycles)

4

In [4]:
# First Sensor, First cycle
cycles[0][0].grip.head()

108000    0.037666
108001    0.012989
108002   -0.025789
108003   -0.062451
108004   -0.078099
Name: 0, dtype: float64

In [5]:
# First Sensor, First cycle
cycles[0][0].flexion.head()

48000   -0.076478
48001   -0.076115
48002   -0.048953
48003   -0.003499
48004    0.039958
Name: 0, dtype: float64

# Generate Dataset based on feature extractors

1. SampEn - Sample Entropy
2. RMS - Root Mean Square
3. WL - Waveform Length
4. WAMP - Willison Amplitude
5. ApEn - Approximate entropy
6. MAV - Mean Absolute Value

Each cycle will be interpreted as a new data acquisition
3 Datasets, one with 299ms, other with 500ms, other with 1 second.

Usar janela de 2(melhor tempo escolhido) segundos para saber qual o melhor extrator,
verificar o método de Onset para ver se a marcação é confiável ou não. FAzer estudo a parte.

Fazer primeiro o feijão com arroz, Usar o tempo base para entender. Checar o onset primeiro

Usar KNN e SVM pois os dados são poucos.

Dataset 299ms:
| SENSOR_0_RMS   |  SENSOR_1_RMS | SENSOR_2_RMS | SENSOR_3_RMS| SENSOR_0_SAMPEN   |  SENSOR_1_SAMPEN | SENSOR_2_SAMPEN | SENSOR_3_SAMPEN |...| CLASS | 
| --- | ---| ---| ---| ---| ---| ---| ---| ---| ---|
| 12 | 13 | 45 | 4 | 5 | 23 | 123 | 123|...|GRIP|


TODO: Do we use the onset algorithm?

IDEIA:

Usar dois dados rápidos para análise rápida e depois no meio do movimento da mão verificar a análise com mais tempo e corrigir o movimento

In [6]:
extractor = ext.RecordExtractor()
cycles = extractor.read_sample(fbase)
np.array(cycles).shape

(4, 5)

In [7]:
import feature_extractors as fe

In [8]:
def populatedict(data, name, dic:dict):
    data = np.array(data)
    if not dic.get(name+"_RMS"): dic[name+"_RMS"] = []
    if not dic.get(name+"_WAVELEN"): dic[name+"_WAVELEN"] = []
    if not dic.get(name+"_WAMP"): dic[name+"_WAMP"] = []
    if not dic.get(name+"_APPEN"): dic[name+"_APPEN"] = []
    if not dic.get(name+"_SAMPEN"): dic[name+"_SAMPEN"] = []
    if not dic.get(name+"_MAV"): dic[name+"_MAV"] = []

    dic[name+"_RMS"].append(fe.rms(data))
    dic[name+"_WAVELEN"].append(fe.waveformlen(data))
    dic[name+"_WAMP"].append(fe.wamp(data))
    dic[name+"_APPEN"].append(fe.app_entropy(data))
    dic[name+"_SAMPEN"].append(fe.sampen(data))
    dic[name+"_MAV"].append(fe.mav(data))

In [9]:
newds = {}
newds_offsetted = {}
newds["class"] = []
newds_offsetted["class"] = []

for j in range(4):
    for i in range(5):
        lower_limit = int(extractor.to_ms(100))
        upper_limit = int(extractor.to_ms(299)) # TODO: Create two databases, one with the offset and the other without

        populatedict(cycles[j][i].rest[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].rest[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].extension[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].extension[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].flexion[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].flexion[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].ulnar_deviation[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].ulnar_deviation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].radial_deviation[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].radial_deviation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].grip[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].grip[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].finger_abduction[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].finger_abduction[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].finger_adduction[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].finger_adduction[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].supination[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].supination[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].pronation[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].pronation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

for i in range(5):
    newds["class"].append("rest")
    newds_offsetted["class"].append("rest")
    
for i in range(5):
    newds["class"].append("extension")
    newds_offsetted["class"].append("extension")

for i in range(5):
    newds["class"].append("flexion")
    newds_offsetted["class"].append("flexion")

for i in range(5):
    newds["class"].append("ulnar_deviation")
    newds_offsetted["class"].append("ulnar_deviation")

for i in range(5):
    newds["class"].append("radial_deviation")
    newds_offsetted["class"].append("radial_deviation")

for i in range(5):
    newds["class"].append("grip")
    newds_offsetted["class"].append("grip")

for i in range(5):
    newds["class"].append("finger_abduction")
    newds_offsetted["class"].append("finger_abduction")

for i in range(5):
    newds["class"].append("finger_adduction")
    newds_offsetted["class"].append("finger_adduction")

for i in range(5):
    newds["class"].append("supination")
    newds_offsetted["class"].append("supination")

for i in range(5):
    newds["class"].append("pronation")
    newds_offsetted["class"].append("pronation")

print(len(newds["class"]))
print(len(newds["SENSOR0_MAV"]))

50
50


In [17]:
pd.DataFrame(newds).head()

,class,SENSOR0_RMS,SENSOR0_WAVELEN,SENSOR0_WAMP,SENSOR0_APPEN,SENSOR0_SAMPEN,SENSOR0_MAV,SENSOR1_RMS,SENSOR1_WAVELEN,SENSOR1_WAMP,...,SENSOR2_WAMP,SENSOR2_APPEN,SENSOR2_SAMPEN,SENSOR2_MAV,SENSOR3_RMS,SENSOR3_WAVELEN,SENSOR3_WAMP,SENSOR3_APPEN,SENSOR3_SAMPEN,SENSOR3_MAV
0,rest,0.031721,5.290727,183,0.687755,0.673635,0.022882,0.032332,6.344441,239,...,33,0.827534,0.790555,0.008300,0.006577,1.331569,14,0.692217,0.694896,0.004631
1,rest,0.070548,11.157630,285,0.580385,0.467871,0.044598,0.087475,13.830049,287,...,138,0.507690,0.379388,0.021779,0.005762,1.052785,3,0.665619,0.592555,0.004100
2,rest,0.027455,4.663072,160,0.717929,0.650282,0.019970,0.031108,5.117540,177,...,37,0.708882,0.540071,0.008646,0.008711,1.521693,21,0.588848,0.555126,0.005797
3,rest,0.029670,5.420028,221,0.806095,0.838035,0.022624,0.036680,7.217883,258,...,28,0.968759,1.016914,0.007080,0.002769,0.558625,0,0.727780,0.771089,0.002001
4,rest,0.027632,4.863877,192,0.773586,0.799442,0.021730,0.030898,6.184191,231,...,54,0.905771,0.906427,0.010075,0.002259,0.551961,0,0.899880,0.991343,0.001859


In [11]:
pd.DataFrame(newds).head(10)

,class,SENSOR0_RMS,SENSOR0_WAVELEN,SENSOR0_WAMP,SENSOR0_APPEN,SENSOR0_SAMPEN,SENSOR0_MAV,SENSOR1_RMS,SENSOR1_WAVELEN,SENSOR1_WAMP,...,SENSOR2_WAMP,SENSOR2_APPEN,SENSOR2_SAMPEN,SENSOR2_MAV,SENSOR3_RMS,SENSOR3_WAVELEN,SENSOR3_WAMP,SENSOR3_APPEN,SENSOR3_SAMPEN,SENSOR3_MAV
0,rest,0.031721,5.290727,183,0.687755,0.673635,0.022882,0.032332,6.344441,239,...,33,0.827534,0.790555,0.008300,0.006577,1.331569,14,0.692217,0.694896,0.004631
1,rest,0.070548,11.157630,285,0.580385,0.467871,0.044598,0.087475,13.830049,287,...,138,0.507690,0.379388,0.021779,0.005762,1.052785,3,0.665619,0.592555,0.004100
2,rest,0.027455,4.663072,160,0.717929,0.650282,0.019970,0.031108,5.117540,177,...,37,0.708882,0.540071,0.008646,0.008711,1.521693,21,0.588848,0.555126,0.005797
3,rest,0.029670,5.420028,221,0.806095,0.838035,0.022624,0.036680,7.217883,258,...,28,0.968759,1.016914,0.007080,0.002769,0.558625,0,0.727780,0.771089,0.002001
4,rest,0.027632,4.863877,192,0.773586,0.799442,0.021730,0.030898,6.184191,231,...,54,0.905771,0.906427,0.010075,0.002259,0.551961,0,0.899880,0.991343,0.001859
5,extension,0.025401,4.794994,169,0.802546,0.819637,0.019949,0.032507,6.670871,236,...,38,0.977281,0.913623,0.008109,0.002997,0.683662,0,0.832709,0.866739,0.002394
6,extension,0.031228,5.998265,242,0.852760,0.857308,0.024281,0.039656,8.262643,287,...,62,1.029031,1.098793,0.009095,0.004319,0.868333,0,0.767640,0.793087,0.003196
7,extension,0.040748,8.260454,325,0.887827,0.929754,0.032016,0.055694,10.656382,365,...,113,0.915409,0.899020,0.014855,0.005250,1.104111,0,0.818243,0.867035,0.004200
8,extension,0.025068,4.995668,185,0.834873,0.849186,0.019954,0.033090,7.341752,291,...,53,0.992684,1.045802,0.009363,0.003023,0.691040,0,0.864963,0.944491,0.002383
9,extension,0.023238,5.474555,205,0.922462,0.978904,0.017589,0.026964,5.579715,228,...,67,1.100858,1.257615,0.008165,0.007759,1.599790,10,0.746533,0.774075,0.005682


In [12]:
pd.DataFrame(newds_offsetted).head(10)


,class,SENSOR0_RMS,SENSOR0_WAVELEN,SENSOR0_WAMP,SENSOR0_APPEN,SENSOR0_SAMPEN,SENSOR0_MAV,SENSOR1_RMS,SENSOR1_WAVELEN,SENSOR1_WAMP,...,SENSOR2_WAMP,SENSOR2_APPEN,SENSOR2_SAMPEN,SENSOR2_MAV,SENSOR3_RMS,SENSOR3_WAVELEN,SENSOR3_WAMP,SENSOR3_APPEN,SENSOR3_SAMPEN,SENSOR3_MAV
0,rest,0.025778,4.690946,152,0.761876,0.765203,0.018910,0.028474,5.824146,213,...,24,0.817085,0.769114,0.007545,0.006339,1.351022,6,0.750815,0.806641,0.004854
1,rest,0.114155,19.795009,404,0.722534,0.461799,0.082869,0.132646,25.435204,403,...,270,0.654284,0.347822,0.040934,0.008554,1.499724,15,0.653104,0.511161,0.006082
2,rest,0.019800,3.340922,95,0.705836,0.643829,0.013796,0.027071,3.633526,109,...,19,0.705468,0.613568,0.006553,0.009203,1.658827,28,0.612566,0.538012,0.006183
3,rest,0.051270,7.857570,275,0.654218,0.632517,0.035259,0.050506,9.167656,286,...,65,0.795376,0.764882,0.010221,0.004391,0.746519,4,0.620374,0.622031,0.002760
4,rest,0.028564,4.953125,200,0.770079,0.772722,0.022161,0.030911,6.341121,239,...,50,0.910142,0.903319,0.009642,0.002239,0.578978,0,0.884294,0.981155,0.001818
5,extension,0.026454,4.734910,171,0.810016,0.822698,0.020734,0.033304,6.759244,235,...,37,0.968790,0.900186,0.007943,0.003100,0.672041,0,0.802200,0.811426,0.002440
6,extension,0.032749,6.003387,229,0.779071,0.739402,0.024325,0.039299,8.007702,275,...,46,0.944866,0.992783,0.009676,0.003986,0.800796,1,0.764172,0.764002,0.003030
7,extension,0.038985,8.250864,307,0.863855,0.917822,0.030406,0.054307,10.507411,354,...,109,0.908859,0.906779,0.015507,0.004782,1.013228,0,0.871741,0.946312,0.003861
8,extension,0.024962,4.710896,168,0.822239,0.803474,0.019781,0.028667,6.181283,246,...,58,0.966731,1.035582,0.009519,0.002887,0.654196,0,0.815910,0.890384,0.002282
9,extension,0.022523,5.092468,185,0.936254,0.993886,0.017163,0.025107,5.332780,206,...,60,1.055453,1.208495,0.008161,0.007971,1.630230,10,0.770670,0.797096,0.006031


### Code encapsulated on `dataset_creation.py`

In [13]:
import dataset_creation as dtc

ds, offsetted_ds = dtc.create_dataset(fbase)

In [16]:
ds.head(10)

,class,SENSOR0_RMS,SENSOR0_WAVELEN,SENSOR0_WAMP,SENSOR0_APPEN,SENSOR0_SAMPEN,SENSOR0_MAV,SENSOR1_RMS,SENSOR1_WAVELEN,SENSOR1_WAMP,...,SENSOR2_WAMP,SENSOR2_APPEN,SENSOR2_SAMPEN,SENSOR2_MAV,SENSOR3_RMS,SENSOR3_WAVELEN,SENSOR3_WAMP,SENSOR3_APPEN,SENSOR3_SAMPEN,SENSOR3_MAV
0,rest,0.031721,5.290727,183,0.687755,0.673635,0.022882,0.032332,6.344441,239,...,33,0.827534,0.790555,0.008300,0.006577,1.331569,14,0.692217,0.694896,0.004631
1,rest,0.070548,11.157630,285,0.580385,0.467871,0.044598,0.087475,13.830049,287,...,138,0.507690,0.379388,0.021779,0.005762,1.052785,3,0.665619,0.592555,0.004100
2,rest,0.027455,4.663072,160,0.717929,0.650282,0.019970,0.031108,5.117540,177,...,37,0.708882,0.540071,0.008646,0.008711,1.521693,21,0.588848,0.555126,0.005797
3,rest,0.029670,5.420028,221,0.806095,0.838035,0.022624,0.036680,7.217883,258,...,28,0.968759,1.016914,0.007080,0.002769,0.558625,0,0.727780,0.771089,0.002001
4,rest,0.027632,4.863877,192,0.773586,0.799442,0.021730,0.030898,6.184191,231,...,54,0.905771,0.906427,0.010075,0.002259,0.551961,0,0.899880,0.991343,0.001859
5,extension,0.025401,4.794994,169,0.802546,0.819637,0.019949,0.032507,6.670871,236,...,38,0.977281,0.913623,0.008109,0.002997,0.683662,0,0.832709,0.866739,0.002394
6,extension,0.031228,5.998265,242,0.852760,0.857308,0.024281,0.039656,8.262643,287,...,62,1.029031,1.098793,0.009095,0.004319,0.868333,0,0.767640,0.793087,0.003196
7,extension,0.040748,8.260454,325,0.887827,0.929754,0.032016,0.055694,10.656382,365,...,113,0.915409,0.899020,0.014855,0.005250,1.104111,0,0.818243,0.867035,0.004200
8,extension,0.025068,4.995668,185,0.834873,0.849186,0.019954,0.033090,7.341752,291,...,53,0.992684,1.045802,0.009363,0.003023,0.691040,0,0.864963,0.944491,0.002383
9,extension,0.023238,5.474555,205,0.922462,0.978904,0.017589,0.026964,5.579715,228,...,67,1.100858,1.257615,0.008165,0.007759,1.599790,10,0.746533,0.774075,0.005682


## Now import more data

In [19]:
fbase2 = pd.read_csv('./2_filtered.csv', header=None)
fbase3 = pd.read_csv('./3_filtered.csv', header=None)


ds_2, _ = dtc.create_dataset(fbase2)
ds_3, _ = dtc.create_dataset(fbase3)

In [21]:
full_ds = pd.concat([ds, ds_2, ds_3])
full_ds

,class,SENSOR0_RMS,SENSOR0_WAVELEN,SENSOR0_WAMP,SENSOR0_APPEN,SENSOR0_SAMPEN,SENSOR0_MAV,SENSOR1_RMS,SENSOR1_WAVELEN,SENSOR1_WAMP,...,SENSOR2_WAMP,SENSOR2_APPEN,SENSOR2_SAMPEN,SENSOR2_MAV,SENSOR3_RMS,SENSOR3_WAVELEN,SENSOR3_WAMP,SENSOR3_APPEN,SENSOR3_SAMPEN,SENSOR3_MAV
0,rest,0.031721,5.290727,183,0.687755,0.673635,0.022882,0.032332,6.344441,239,...,33,0.827534,0.790555,0.008300,0.006577,1.331569,14,0.692217,0.694896,0.004631
1,rest,0.070548,11.157630,285,0.580385,0.467871,0.044598,0.087475,13.830049,287,...,138,0.507690,0.379388,0.021779,0.005762,1.052785,3,0.665619,0.592555,0.004100
2,rest,0.027455,4.663072,160,0.717929,0.650282,0.019970,0.031108,5.117540,177,...,37,0.708882,0.540071,0.008646,0.008711,1.521693,21,0.588848,0.555126,0.005797
3,rest,0.029670,5.420028,221,0.806095,0.838035,0.022624,0.036680,7.217883,258,...,28,0.968759,1.016914,0.007080,0.002769,0.558625,0,0.727780,0.771089,0.002001
4,rest,0.027632,4.863877,192,0.773586,0.799442,0.021730,0.030898,6.184191,231,...,54,0.905771,0.906427,0.010075,0.002259,0.551961,0,0.899880,0.991343,0.001859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,pronation,0.030206,5.518621,191,0.751244,0.761140,0.022158,0.040091,8.376067,303,...,17,0.661933,0.606462,0.016791,0.004540,0.886658,0,0.744321,0.719639,0.003487
46,pronation,0.039847,6.938194,257,0.805650,0.800657,0.030765,0.039590,8.144405,288,...,6,0.536482,0.487924,0.018160,0.002153,0.424398,0,0.800815,0.827587,0.001619
47,pronation,0.115721,22.247809,503,0.869731,0.916870,0.089609,0.119101,27.785782,502,...,121,0.725848,0.708992,0.024602,0.009298,1.950349,17,0.761951,0.818972,0.007312
48,pronation,0.042113,6.958032,267,0.747249,0.730523,0.031823,0.049149,9.963021,329,...,35,0.675932,0.637735,0.016463,0.003871,0.759703,1,0.648493,0.674529,0.002837


In [23]:
full_ds.columns

Index(['class', 'SENSOR0_RMS', 'SENSOR0_WAVELEN', 'SENSOR0_WAMP',
       'SENSOR0_APPEN', 'SENSOR0_SAMPEN', 'SENSOR0_MAV', 'SENSOR1_RMS',
       'SENSOR1_WAVELEN', 'SENSOR1_WAMP', 'SENSOR1_APPEN', 'SENSOR1_SAMPEN',
       'SENSOR1_MAV', 'SENSOR2_RMS', 'SENSOR2_WAVELEN', 'SENSOR2_WAMP',
       'SENSOR2_APPEN', 'SENSOR2_SAMPEN', 'SENSOR2_MAV', 'SENSOR3_RMS',
       'SENSOR3_WAVELEN', 'SENSOR3_WAMP', 'SENSOR3_APPEN', 'SENSOR3_SAMPEN',
       'SENSOR3_MAV'],
      dtype='object')

In [38]:
import numpy as np

import sklearn as sk

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.neighbors import KNeighborsClassifier

## RMS Test

In [24]:
class_column = "class"
train_columns = [f"SENSOR{i}_RMS" for i in range(4)]

print(train_columns)

['SENSOR0_RMS', 'SENSOR1_RMS', 'SENSOR2_RMS', 'SENSOR3_RMS']


In [40]:
X = full_ds[train_columns]
Y = full_ds[class_column]

In [41]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.8,test_size=0.2)

In [39]:
model = KNeighborsClassifier()
model.fit(X_train, Y_train)

KNeighborsClassifier()

In [45]:
preds_valid = model.predict(X_test)
score_valid = mean_absolute_error(Y_test, preds_valid)

ValueError: could not convert string to float: 'finger_adduction'